# Theoretical Questions

* This is the theoretical part of the final project. It includes theoretical questions from various topics covered in the course.
* There are 7 questions among which you need to choose 6, according to the following key:
    + Question 1 is **mandatory**.
    + Choose **one question** from questions 2-3.
    + Question 4 is **mandatory**.
    + Questions 5-6 are **mandatory**.
    + Question 7 is **mandatory**.
* Question 1 is worth 15 points, whereas the other questions worth 7 points.
* All in all, the maximal grade for this parts is 15+7*5=50 points.
* **You should answer the questions on your own. We will check for plagiarism.**
* If you need to add external images (such as graphs) to this notebook, please put them inside the 'imgs' folder. DO NOT put a reference to an external link.
* Good luck!

## Part 1: General understanding of the course material

### Question 1

1.  Relate the number of parameters in a neural network to the over-fitting phenomenon (*).
    Relate this to the design of convolutional neural networks, and explain why CNNs are a plausible choice for an hypothesis class for visual classification tasks.

    (*) In the context of classical under-fitting/over-fitting in machine learning models.

CNNs are a plausible choice for an hypothesis class for visual classification tasks and avoid over-fitting because:

- CNNs use shared weights across convolutional layers to recognize fundamental characteristics (edges, corners, and textures) independently of where they are in the picture. This technique is known as "hared parameter learning." In comparison to fully connected layers, this drastically reduces the number of parameters, which makes the network less prone to overfitting, especially in situations when there is little training data.
- CNNs are constructed with a number of layers that extract more complicated features. Higher layers combine these elements from lower layers to depict more complex patterns. Lower layers collect fundamental features. This hierarchical feature extraction assists the network in concentrating on pertinent data while ignoring minute differences that can result in overfitting.
- CNNs frequently use pooling layers to reduce the spatial dimensions of feature maps by downsampling them. The network can more easily generalize to changes in object location, size, and orientation thanks to this procedure, which creates a type of spatial invariance.
- As a result of weight sharing and pooling, CNNs are made to be translation-invariant. This characteristic is advantageous for visual tasks in which the identification of an item is unaffected by its location within a picture.

This constraint on the search space of the network prevents over-fitting as the number of parameters increases..ntity.

2. Consider the linear classifier model with hand-crafted features: 
    $$f_{w,b}(x) = w^T \psi(x) + b$$
    where $x \in \mathbb{R}^2$, $\psi$ is a non-learnable feature extractor and assume that the classification is done by $sign(f_{w,b}(x))$. Let $\psi$ be the following feature extractor $\psi(x)=x^TQx$ where $Q \in \mathbb{R}^{2 \times 2}$ is a non-learnable positive definite matrix. Describe a distribution of the data which the model is able to approximate, but the simple linear model fails to approximate (hint: first, try to describe the decision boundary of the above classifier).

A distribution of the data which the model is able to approximate, but the simple linear model fails to approximate is Annular Data Distribution

Imagine a dataset with points grouped in circular rings of varied widths and following an annular distribution in a 2D space. Without feature extraction, a straightforward linear classifier will fail to accurately divide the classes using a single straight line. However, by applying feature extraction with the matrix, we can transform the data so that a linear classifier in the transformed space can roughly approach a circular decision boundary. 

3. Assume that we would like to train a Neural Network for classifying images into $C$ classes. Assume that the architecture can be stored in the memory as a computational graph with $N$ nodes where the output is the logits (namely, before applying softmax) for the current batch ($f_w: B \times Ch \times H \times W \rightarrow B \times C$). Assume that the computational graph operates on *tensor* values.
    * Implement the CE loss assuming that the labels $y$ are hard labels given in a LongTensor (as usual). **Use Torch's log_softmax and index_select functions** and implement with less as possible operations.

In [ ]:
from torch.nn.functional import log_softmax
from torch import index_select
# Input:  model, x, y. 
# Output: the loss on the current batch.
logits = model(x)
...
loss = -log_softmax(logits, dim=1).index_select(-1,y).diag().sum()

* Using the model's function as a black box, draw the computational graph (treating both log_softmax and index_select as an atomic operations). How many nodes are there in the computational graph?

x -> model -> log_softmax -> index_select -> sum -> loss

There are 4 nodes

* Now, instead of using hard labels, assume that the labels are representing some probability distribution over the $C$ classes. How would the gradient computation be affected? analyze the growth in the computational graph, memory and computation.

**Gradient Computation:**

When the labels are representing some probability distribution over the $C$ classes (probability distributions), both the predicted distribution and the target distribution are used in the gradient computation. The difference between these distributions is taken into account by the gradient update. Therefore, the gradient updates depend on how well the projected probabilities match the target distribution, rather than only on whether or not the prediction is accurate.

**Computational Complexity and Memory:**

This can lead to increased in computation and memory usage because:

1. **Divergence Computation:** As opposed to the conventional cross-entropy loss with hard labels, calculating divergences (such as the KL divergence) between projected probability and target probabilities requires extra calculations. We would have to multiply each log_softmax by its corresponding target distribution, which would require extra processing.

2. **Memory Usage:** When using labels with probability distributions, it is necessary to store probability distributions rather than just single numbers for each sample. Higher memory utilization might come from this, especially for bigger datasets.

3. **Additional Gradients:** Additional gradients must be calculated during backpropagation for both predicted probability (p) and target probabilities (q). Additional memory and processing power are needed for this.

**Growth in Computational Graph:**

Divergence calculations, which cover terms pertaining to both the anticipated and target distributions, add more terms to the computational graph, making it more complex. When there are more classes, the graph's complexity increase.



* Apply the same analysis in the case that we would like to double the batch size. How should we change the learning rate of the optimizer?

**Gradient Computation:**
The gradients calculated in each iteration will be based on a bigger subset of the dataset when the batch size is doubled. As a result, gradient estimations may be more precise and convergence may be more stable. 

**Memory Usage:**
For both forward and backward passes, doubling the batch size increases the amount of memory needed to store the intermediate activations and gradients. This is because more data must be processed on each cycle, which calls for more memory.

**Growth in Computational Graph:**
No growth in computational graph and complexity.

**Learning Rate Adjustment:**
The learning rate should normally be adjusted to maintain a constant rate of parameter updates and convergence when the batch size is doubled. To get the same impact on the model's weights, the learning rate should be scaled proportionally to the batch size.

## Part 2: Optimization & Automatic Differentiation

### Question 2: resolving gradient conflicts in multi-task learning

Assume that you want to train a model to perform two tasks: task 1 and task 2. 
For each such task $i$ you have an already implemented function *loss\_i = forward_and_compute_loss_i(model,inputs)* such that given the model and the inputs it computes the loss w.r.t task $i$ (assume that the computational graph is properly constructed). We would like to train our model using SGD to succeed in both tasks as follows: in each training iteration (batch) -
* Let $g_i$ be the gradient w.r.t the $i$-th task.
* If $g_1 \cdot g_2 < 0$:
    + Pick a task $i$ at random.
    + Apply GD w.r.t only that task.
* Otherwise:
    + Apply GD w.r.t both tasks (namely $\mathcal{L}_1 + \mathcal{L}_2$).

Note that in the above formulation the gradient is a thought of as a concatination of all the gradient w.r.t all the models parameters, and $g_1 \cdot g_2$ stands for a dot product.

What parts should be modified to implement the above? Is it the optimizer, the training loop or both? Implement the above algorithm in a code cell/s below

In [ ]:
 # We only need to modify the training loop
for epoch in range(num_epochs):
    for batch in dataloader:
        inputs = batch['inputs']
        
        # Compute gradients for both tasks
        optimizer.zero_grad()
        loss_1 = forward_and_compute_loss_1(model, inputs)
        loss_2 = forward_and_compute_loss_2(model, inputs)
        loss_total = loss_1 + loss_2
        # Compute dot product of gradients
        optimizer.zero_grad()
        loss_1.backward()
        grad_1 = torch.cat([p.grad.view(-1) for p in model.parameters()])
        optimizer.zero_grad()
        loss_2.backward()
        grad_2 = torch.cat([p.grad.view(-1) for p in model.parameters()])
        dot_product = torch.dot(grad_1, grad_2)
        if dot_product < 0:
            # Pick a task at random
            if torch.rand(1) < 0.5:
                optimizer.zero_grad()
                loss_1.backward()
                optimizer.step()
            else:
                # Apply GD w.r.t only the other task
                optimizer.zero_grad()
                loss_2.backward()
                optimizer.step()
        else:
            # Apply GD w.r.t both tasks
            optimizer.zero_grad()
            loss_total.backward()
            optimizer.step()
        

### Question 3: manual automatic differentiation

Consider the following two-input two-output function:
$$ f(x,y) = (x^2\sin(xy+\frac{\pi}{2}), x^2\ln(1+xy)) $$
* Draw a computational graph for the above function. Assume that the unary atomic units are squaring, taking square root, $\exp,\ln$, basic trigonometric functions and the binary atomic units are addition and multiplication. You would have to use constant nodes.
* Calculate manually the forward pass.
* Calculate manually the derivative of all outputs w.r.t all inputs using a forward mode AD.
* Calculate manually the derivative of all outputs w.r.t all inputs using a backward mode AD.

## Part 3: Sequential Models

### Question 4: RNNs vs Transformers in the real life

In each one of the following scenarios decide whether to use RNN based model or a transformer based model. Justify your choice.
1. You are running a start-up in the area of automatic summarization of academic papers. The inference of the model is done on the server side, and it is very important for it to be fast.
2. You need to design a mobile application that gathers small amount of data from few apps in every second and then uses a NN to possibly generate an alert given the information in the current second and the information from the past minute.
3. You have a prediction task over fixed length sequences on which you know the following properties:
    + In each sequence there are only few tokens that the model should attend to.
    + Most of the information needed for generating a reliable prediction is located at the beginning of the sequence.
    + There is no restriction on the computational resources.

Scenario 1: Automatic Summarization of Academic Papers

A transformer-based approach might be better appropriate for this situation.
Transformers can efficiently extract contextual information from various document sections, which is crucial for producing reliable and cogent summaries.

Scenario 2: Mobile Application for Real-time Data and Alerts

RNN-based model, like as LSTM or GRU, might be more suited in this situation. RNNs can capture temporal relationships in the data and intuitively handle sequential data.
RNNs can effectively analyze this temporal component since the program collects data every second and leverages data from the previous minute.
Compared to transformers, RNNs have fewer parameters, which is beneficial for real-time processing on a mobile device with constrained resources.

Scenario 3: Prediction with Fixed Length Sequences

An RNN-based model appears to be appropriate for this circumstance based on the information supplied.
The small number of important tokens in each sequence emphasizes the need for an attention mechanism, which RNNs may offer.
The sequence's beginning contains the majority of the data required for precise predictions, which is consistent with RNNs' capacity to understand the sequence's beginning.

## Part 4: Generative modeling

### Question 5: VAEs and GANS

Suggest a method for combining VAEs and GANs. Focus on the different components of the model and how to train them jointly (the objectives). Which drawbacks of these models the combined model may overcome? which not?


A method for combining VAEs and GANs is the AAEs:

1. **Architecture:**
   - AAEs combine the adversarial training of GANs with the encoder and decoder structure of VAEs.
   - Similar to a VAE, the encoder maps input data to a latent space.
   - Similar to a VAE, the generator (decoder) creates data samples from latent codes.
   - To discriminate between latent codes taken from the data distribution and those from the encoder, a discriminator, as used in GANs, is introduced instead of using the KL divergence.

2. **Prior Matching:**
    - AAEs use the discriminator to match the approximate posterior to the prior of the genuine data distribution rather than forcing the approximate posterior (encoder output) to match a unit Gaussian. This enables the AAE to learn a continuous and more precise representation of latent space.

3. **Adversarial Training:**
   - The discriminator is trained to distinguish between samples from the aggregated approximate posterior (VAE encoder) and samples from the true data latent code distribution.

Drawbacks Addressed by AAEs:

1. **Latent Space Quality:**
    - AAEs improve the continuous and smooth nature of the learnt latent space to address the issue of blurry or low-quality samples produced by VAEs. AAEs make sure that the latent space better captures the data manifold by comparing the aggregated posterior to the prior of the real data distribution.

2. **Data Distribution Capture:**
    - By training the model to match the previous distribution of the true data distribution, AAEs overcome the shortcoming of VAEs in capturing complicated data distributions. This allows the AAE to more correctly identify the modes and fluctuations in the data distribution, which improves sample quality.

Drawbacks AAEs Might Not Fully Overcome:

1. **Inference Speed:**
    -  AAEs and VAEs both use encoder-decoder structures, which may not be able to meet the theoretical speed requirement indicated in some cases.
    - In situations demanding quick inference, transformers—known for their parallelization abilities—might nevertheless perform better than both VAEs and AAEs.


### Question 6: Diffusion Models

Show that $q(x_{t-1}|x_t,x_0)$ is tractable and is given by $\mathcal{N}(x_{t-1};\tilde{\mu}(x_t,x_0),\tilde{\beta_t}I)$ where the terms for $\tilde{\mu}(x_t,x_0)$ and $\tilde{\beta_t}$ are given in the last tutorial. Do so by explicitly computing the PDF.

We create a forward noise process $q$ that, given a data distribution $x_0 sim q(x_0)$, generates latents $x_1$ through $x_T$ by adding Gaussian noise at time $t$ with variance $beta_t \in (0, 1)$ as follows:
$$
q(x_1, ..., x_T | x_0) = \prod_{t=1}^{T} q(x_t | x_{t-1})
$$

$$
q(x_t | x_{t-1}) = \mathcal{N} \left( x_t; (1 - \beta_t) x_{t-1}, \beta_t I \right)
$$


* The forward process allow us to compute $q(x_t|x_0)$ directly:
$$ q(x_t|x_0) = \mathcal{N}(x_t;\sqrt{\bar{\alpha_t}}x_0, (1-\bar{\alpha_t})\mathbf{I})$$
* where $\alpha_t := 1- \beta_t$ and $\bar{\alpha_t} = \prod_{t=1}^T \alpha_t$.

Using Bayes' theorem, we compute the posterior $q(x_t-1|x_t, x_0)$:
   
   $$p(x_{t-1}|x_t, x_0) = \frac{p(x_t, x_{t-1} | x_0)}{p(x_t | x_0)}$$

   $$p(x_{t-1}|x_t, x_0) = \frac{q(x_t | x_{t-1}, x_0) \cdot p(x_{t-1} | x_0)}{p(x_t | x_0)}$$
Using markov assumption $ q(x_t | x_{t-1}, x_0) = q(x_t | x_{t-1}) $ we get:
   $$p(x_{t-1}|x_t, x_0) = \frac{q(x_t | x_{t-1}) \cdot p(x_{t-1} | x_0)}{p(x_t | x_0)}$$
   $$q(x_{t-1}|x_t, x_0) = \frac{\mathcal{N}(x_t; \tilde{\mu}_t(x_{t-1}, x_0), \beta_t I) \cdot \mathcal{N}(x_{t-1}; \sqrt{\bar{\alpha}_{t-1}} x_0, (1 - \bar{\alpha}_{t-1}) I)}{\mathcal{N}(x_t; \sqrt{\bar{\alpha}_t} x_0, (1 - \bar{\alpha}_t) I)}$$
   
The resulting posterior distribution is also a Gaussian distribution.

As a result we get

$$ q(x_{t-1}|x_t,x_0) = \mathcal{N}(x_{t-1}; \tilde{\mu}(x_t,x_0),\tilde{\beta_t}\mathbf{I})$$

where:

$$ \tilde{\mu}(x_t,x_0) := \frac{\sqrt{\bar{\alpha}_{t-1}}\beta_t}{1-\bar{\alpha}_t}x_0 + \frac{\sqrt{\alpha_t}(1-\bar{\alpha}_{t-1})}{1-\bar{\alpha}_t}x_t$$

and:
$$\tilde{\beta}_t := \frac{1-\bar{\alpha}_{t-1}}{1-\bar{\alpha}_t} \beta_t$$

## Part 5: Training Methods

### Question 7: Batch Normalization and Dropout

For both BatchNorm and Dropout analyze the following:
1. How to use them during the training phase (both in forward pass and backward pass)?
2. How differently they behave in the inference phase? How to distinguish these operation modes in code?
3. Assume you would like to perform multi-GPU training (*) to train your model. What should be done in order for BatchNorm and dropout to work properly? assume that each process holds its own copy of the model and that the processes can share information with each other.

(*): In a multi-GPU training each GPU is associated with its own process that holds an independent copy of the model. In each training iteration a (large) batch is split among these processes (GPUs) which compute the gradients of the loss w.r.t the relevant split of the data. Afterwards, the gradients from each process are then shared and averaged so that the GD would take into account the correct gradient and to assure synchornization of the model copies. Note that the proccesses are blocked between training iterations.

1. To use them during the training phase:

    **BatchNorm**
    - Forward Pass: Activations within each mini-batch are normalized by BatchNorm during training, which centers them around zero and scales them using the learnt parameters (mean and variance).
    - Backward Pass: During backpropagation, gradients propagate via normalized activations and update the mean, variance, and model weights.
  
   **Dropout**
    - Forward Pass: Dropout randomly deactivates neurons with a predetermined probability during training to minimize co-dependencies and avoid overfitting.
    - Backward Pass: During backpropagation, gradients only pass via neurons that are activated.
  
2.   Different behavior in the inference phase:
    - Dropout is disabled and all neurons are engaged during inference. To take training-time dropout into account, outputs are scaled by dropout probability. while Batch Norm is still active during inference.
    - To distinguish these operation modes use model.eval() for inference and model.train() for training.
4. For proper BatchNorm performance in multi-GPU configurations we need to independently calculate statistics on each GPU, sync statistics between GPUs and perform BatchNorm across all GPU. To guarantee consistency in the results for Drop out, ensure consistent dropout probabilities across all GPUs.rs.